In [ ]:
import numpy as np

def get_initial_superposition(ket_0, h_gate):
    """
    Step 1: Initialize to equal superposition |s>.
    Start with |00> and apply H to both qubits.
    """
    # |00> = |0> tensor |0>
    psi_0 = np.kron(ket_0, ket_0)

    # H_total = H tensor H
    h_total = np.kron(h_gate, h_gate)

    # |s> = H_total * |00>
    psi_s = h_total @ psi_0

    print(f"Initial Superposition |s> (Uniform Probabilities):\n{psi_s.flatten().round(3)}")
    return psi_s

def create_oracle_matrix(target_index):
    """
    Step 2: The Oracle U_w.
    It flips the phase (sign) of the target state |w> only.
    Mathematically: Identity matrix, but Index[target, target] = -1
    """
    # Start with 4x4 Identity
    U_w = np.eye(4)

    # Flip the sign of the target element
    U_w[target_index, target_index] = -1

    return U_w

def create_diffuser_matrix():
    """
    Step 3: The Diffuser (Amplitude Amplification).
    Operator D = 2|s><s| - I
    This 'inverts' amplitudes about the mean.
    """
    # |s> vector for 2 qubits is [0.5, 0.5, 0.5, 0.5]^T
    s_vector = np.array([[0.5], [0.5], [0.5], [0.5]])

    # Outer product |s><s| creates a matrix where every element is 0.25
    s_outer_s = s_vector @ s_vector.T

    # Identity matrix
    I = np.eye(4)

    # D = 2 * (|s><s|) - I
    diffuser = 2 * s_outer_s - I

    return diffuser

def measure_state(psi):
    """
    Calculates probabilities for all 4 basis states.
    """
    probs = np.abs(psi.flatten())**2
    return probs

def run_grover_simulation(target_name, target_index, ket_0, h_gate):
    """
    Orchestrates the Grover Search.
    """
    print(f"\n{'='*60}")
    print(f"--- Searching for Target: {target_name} (Index {target_index}) ---")

    # 1. Initialization (|s>)
    psi = get_initial_superposition(ket_0, h_gate)

    # 2. Apply Oracle (Mark the winner)
    oracle = create_oracle_matrix(target_index)
    psi = oracle @ psi
    print(f"\nState after Oracle (Phase flipped for {target_name}):\n{psi.flatten().round(3)}")

    # 3. Apply Diffuser (Amplify the winner)
    diffuser = create_diffuser_matrix()
    psi = diffuser @ psi
    print(f"\nState after Diffuser (Amplitude Amplification):\n{psi.flatten().round(3)}")

    # 4. Measure
    probs = measure_state(psi)

    # Find the index with the highest probability
    measured_index = np.argmax(probs)
    measured_prob = probs[measured_index]

    print(f"\nFinal Probabilities: {probs.round(3)}")
    print(f">> FOUND: Index {measured_index} with probability {measured_prob*100:.1f}%")

    if measured_index == target_index:
        print(">> SUCCESS: Grover's algorithm found the target!")
    else:
        print(">> FAILURE: Algorithm missed.")

def main():
    # --- Configuration ---
    # |0>
    ket_0 = np.array([[1], [0]])

    # H Gate
    h_gate = (1 / np.sqrt(2)) * np.array([[1, 1], [1, -1]])

    # Define Targets (Index 0 to 3)
    # 0 = |00>, 1 = |01>, 2 = |10>, 3 = |11>
    targets = [
        ("|11>", 3), # Let's search for the last item
        ("|01>", 1)  # Let's search for the second item
    ]

    for name, idx in targets:
        run_grover_simulation(name, idx, ket_0, h_gate)

if __name__ == "__main__":
    main()


--- Searching for Target: |11> (Index 3) ---
Initial Superposition |s> (Uniform Probabilities):
[0.5 0.5 0.5 0.5]

State after Oracle (Phase flipped for |11>):
[ 0.5  0.5  0.5 -0.5]

State after Diffuser (Amplitude Amplification):
[0. 0. 0. 1.]

Final Probabilities: [0. 0. 0. 1.]
>> FOUND: Index 3 with probability 100.0%
>> SUCCESS: Grover's algorithm found the target!

--- Searching for Target: |01> (Index 1) ---
Initial Superposition |s> (Uniform Probabilities):
[0.5 0.5 0.5 0.5]

State after Oracle (Phase flipped for |01>):
[ 0.5 -0.5  0.5  0.5]

State after Diffuser (Amplitude Amplification):
[0. 1. 0. 0.]

Final Probabilities: [0. 1. 0. 0.]
>> FOUND: Index 1 with probability 100.0%
>> SUCCESS: Grover's algorithm found the target!
